Import 

In [92]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn import model_selection
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split,GridSearchCV,RandomizedSearchCV
from sklearn import metrics
import pandas_datareader.data as web
from time import time
from sklearn.model_selection import KFold

Data preparation

In [93]:
# Get Data
# Read stock data use pandas_datareader.data from web
# Get Stock Data

stk_tickers = ['MSFT', 'IBM', 'GOOGL']
ccy_tickers = ['DEXJPUS', 'DEXUSUK']
idx_tickers = ['SP500', 'DJIA', 'VIXCLS']
stk_data = web.DataReader(stk_tickers, 'yahoo')
ccy_data = web.DataReader(ccy_tickers, 'fred')
idx_data = web.DataReader(idx_tickers, 'fred')
# Select columns
base = stk_data.loc[:, ('Adj Close', 'MSFT')]
X1 = stk_data.loc[:, ('Adj Close', ('GOOGL', 'IBM'))]
X2 = ccy_data
X3 = idx_data

# Standardize Data
# Standardized data (X1, X2, X3) with kept index (date)
standard_scaler = preprocessing.StandardScaler()
std = standard_scaler.fit_transform(X1.values)
X1 = pd.DataFrame(data=std,index=X1.index,columns=X1.columns)
X1

Attributes Adj Close          
Symbols        GOOGL       IBM
Date                          
2017-09-22 -1.143638 -0.560756
2017-09-25 -1.157853 -0.509278
2017-09-26 -1.152867 -0.461280
2017-09-27 -1.117297 -0.523887
2017-09-28 -1.109524 -0.523887
...              ...       ...
2022-09-15  0.621321  0.925725
2022-09-16  0.617838  1.090012
2022-09-19  0.626387  1.132469
2022-09-20  0.565282  1.000485
2022-09-21  0.506394  0.874039

[1258 rows x 2 columns]

In [94]:
# Calculate ความแตกต่างของค่า ราคา 'Adj Close', 'MSFT’)ย้อนหลัง backHisotry วัน
backHistory = [30, 45, 60, 90, 180, 240] # -> ทดลองหยิบ 3 ค่า 3 รูปแบบ เพื่อดูระยะเวลาการดูค่าข้อมูลย้อนหลงัหลายๆแบบและเปรียบเทียบ MSE
BH1, BH2, BH3 = backHistory[1], backHistory[3], backHistory[4]
return_period = 2
Y = base.shift(-return_period)
X4_BH1 = base.diff( BH1).shift( - BH1)
X4_BH2 = base.diff( BH2).shift( - BH2)
X4_BH3 = base.diff( BH3).shift( - BH3)
X4 = pd.concat([X4_BH1, X4_BH2, X4_BH3], axis=1)
X4.columns = ['MSFT_3DT', 'MSFT_6DT', 'MSFT_12DT']
X4 = pd.DataFrame(standard_scaler.fit_transform(X4.values), index = X4.index,columns=X4.columns)

In [95]:
# Forming Dataset
X = pd.concat([X1, X2, X3, X4], axis=1)
data = pd.concat([Y, X], axis=1)
data

,"(Adj Close, MSFT)","(Adj Close, GOOGL)","(Adj Close, IBM)",DEXJPUS,DEXUSUK,SP500,DJIA,VIXCLS,MSFT_3DT,MSFT_6DT,MSFT_12DT
2017-09-22,68.838745,-1.143638,-0.560756,NaN,NaN,NaN,NaN,NaN,0.119908,0.194477,-0.171718
2017-09-25,69.393127,-1.157853,-0.509278,111.64,1.3457,2496.66,22296.09,10.21,0.237056,0.143192,-0.153079
2017-09-26,69.411926,-1.152867,-0.461280,112.16,1.3422,2496.84,22284.32,10.17,0.153312,-0.001935,-0.137317
2017-09-27,69.994499,-1.117297,-0.523887,112.76,1.3401,2507.04,22340.71,9.87,0.166523,0.103378,-0.189077
2017-09-28,70.107254,-1.109524,-0.523887,112.66,1.3435,2510.06,22381.20,9.55,0.170334,0.036576,-0.169436
...,...,...,...,...,...,...,...,...,...,...,...
2022-09-15,244.520004,0.621321,0.925725,143.57,1.1472,3901.35,30961.82,26.27,NaN,NaN,NaN
2022-09-16,242.449997,0.617838,1.090012,143.04,1.1419,3873.33,30822.42,26.30,NaN,NaN,NaN
2022-09-19,238.949997,0.626387,1.132469,NaN,NaN,3899.89,31019.68,25.76,NaN,NaN,NaN
2022-09-20,NaN,0.565282,1.000485,NaN,NaN,3855.93,30706.23,27.16,NaN,NaN,NaN


In [96]:
# Drop NA
data.dropna(inplace=True)
# View Statistics
data.describe()

,"(Adj Close, MSFT)","(Adj Close, GOOGL)","(Adj Close, IBM)",DEXJPUS,DEXUSUK,SP500,DJIA,VIXCLS,MSFT_3DT,MSFT_6DT,MSFT_12DT
count,1060.000000,1060.000000,1060.000000,1060.000000,1060.000000,1060.000000,1060.000000,1060.000000,1060.000000,1060.000000,1060.000000
mean,165.833432,-0.206168,-0.223745,109.250689,1.318665,3244.556321,27812.927142,19.624500,0.128459,0.153403,0.001555
std,73.227022,0.919130,0.872918,2.738197,0.053738,639.726308,3910.742829,8.980875,0.886430,0.870535,0.994364
min,69.393127,-1.157853,-3.286945,102.520000,1.149200,2237.400000,18591.930000,9.140000,-3.837787,-3.767676,-3.700736
25%,102.806959,-0.860913,-0.698383,107.300000,1.283825,2761.265000,25016.967500,13.740000,-0.269065,-0.233211,-0.416738
50%,140.476227,-0.646407,-0.248734,109.345000,1.314050,2975.975000,26473.955000,17.380000,0.089913,0.087077,0.051284
75%,214.626717,0.138314,0.262102,111.222500,1.363175,3668.327500,30003.322500,22.525000,0.531877,0.590334,0.626406
max,340.882812,2.107085,1.898889,115.340000,1.433200,4796.560000,36585.060000,82.690000,2.578861,2.210962,2.431570


In [97]:
# Assign X, Y (drop datetime index)
Y = data[data.columns[0]]
X = data[data.columns[1:]]
print(Y)
print(X)
# feature selection (correlation)
# Calculate correlation between variables for only continuous data columns
corr_data = X.corr()
# Reduce Corr() to Lower Matrix
lower_tri = corr_data.where(np.tril(np.ones(corr_data.shape),k=-1).astype(np.bool))
lower_tri.fillna(0, inplace=True)
# Drop columns if |correlation value| > 0.9
to_drop = [column for column in lower_tri.columns if any(lower_tri[column] > 0.9)]

X.drop(columns=to_drop, inplace=True)
X

2017-09-25     69.393127
2017-09-26     69.411926
2017-09-27     69.994499
2017-09-28     70.107254
2017-09-29     69.778389
                 ...    
2021-12-27    339.730377
2021-12-28    337.117432
2021-12-29    334.136932
2021-12-30    332.577087
2022-01-03    314.326324
Name: (Adj Close, MSFT), Length: 1060, dtype: float64
            (Adj Close, GOOGL)  (Adj Close, IBM)  ...  MSFT_6DT  MSFT_12DT
2017-09-25           -1.157853         -0.509278  ...  0.143192  -0.153079
2017-09-26           -1.152867         -0.461280  ... -0.001935  -0.137317
2017-09-27           -1.117297         -0.523887  ...  0.103378  -0.189077
2017-09-28           -1.109524         -0.523887  ...  0.036576  -0.169436
2017-09-29           -1.095420         -0.564232  ... -0.163721  -0.186342
...                        ...               ...  ...       ...        ...
2021-12-27            2.045917          1.055487  ... -3.188239  -3.498090
2021-12-28            2.007308          1.145359  ... -3.245814  -3.661

C:\Users\napat\AppData\Local\Temp\ipykernel_12840\3771836981.py:10: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  lower_tri = corr_data.where(np.tril(np.ones(corr_data.shape),k=-1).astype(np.bool))
C:\Users\napat\AppData\Local\Temp\ipykernel_12840\3771836981.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.drop(columns=to_drop, inplace=True)


,"(Adj Close, IBM)",DEXJPUS,DEXUSUK,DJIA,VIXCLS,MSFT_3DT,MSFT_6DT,MSFT_12DT
2017-09-25,-0.509278,111.64,1.3457,22296.09,10.21,0.237056,0.143192,-0.153079
2017-09-26,-0.461280,112.16,1.3422,22284.32,10.17,0.153312,-0.001935,-0.137317
2017-09-27,-0.523887,112.76,1.3401,22340.71,9.87,0.166523,0.103378,-0.189077
2017-09-28,-0.523887,112.66,1.3435,22381.20,9.55,0.170334,0.036576,-0.169436
2017-09-29,-0.564232,112.64,1.3402,22405.09,9.51,-0.036140,-0.163721,-0.186342
...,...,...,...,...,...,...,...,...
2021-12-27,1.055487,114.85,1.3438,36302.38,17.68,-2.796915,-3.188239,-3.498090
2021-12-28,1.145359,114.75,1.3432,36398.21,17.54,-2.973052,-3.245814,-3.661982
2021-12-29,1.209429,114.97,1.3475,36488.63,16.95,-3.360509,-3.684912,-3.700736
2021-12-30,1.259260,115.17,1.3500,36398.08,17.33,-3.837787,-3.379544,-3.631296


In [98]:
# Train / Test Preparation (try 2 Option)
# Option#1
test_size = (0.3 * len( X )).__round__()
train_size = (0.7 * len( X )).__round__()
X_train, X_test = X[0:train_size], X[train_size:len(X)]
Y_train, Y_test = Y[0:train_size], Y[train_size:len(X)]
# Option #2
seed = 4
# X_train, X_test, Y_train, Y_test = model_selection.train_test_split(X, Y, test_size=0.3, random_state=seed)
X_train

,"(Adj Close, IBM)",DEXJPUS,DEXUSUK,DJIA,VIXCLS,MSFT_3DT,MSFT_6DT,MSFT_12DT
2017-09-25,-0.509278,111.64,1.3457,22296.09,10.21,0.237056,0.143192,-0.153079
2017-09-26,-0.461280,112.16,1.3422,22284.32,10.17,0.153312,-0.001935,-0.137317
2017-09-27,-0.523887,112.76,1.3401,22340.71,9.87,0.166523,0.103378,-0.189077
2017-09-28,-0.523887,112.66,1.3435,22381.20,9.55,0.170334,0.036576,-0.169436
2017-09-29,-0.564232,112.64,1.3402,22405.09,9.51,-0.036140,-0.163721,-0.186342
...,...,...,...,...,...,...,...,...
2020-09-11,-0.959465,106.17,1.2788,27665.64,26.87,0.291446,0.278194,0.343659
2020-09-14,-0.909166,105.68,1.2876,27993.33,25.85,0.254101,0.262316,0.301435
2020-09-15,-0.881223,105.50,1.2866,27995.60,25.59,-0.093337,0.271082,0.159667
2020-09-16,-0.739112,104.94,1.2980,28032.38,26.04,-0.042263,0.532254,0.411943


GridSearchCV() 

In [99]:
# Create Model List
regression = { 'LR': LinearRegression(), 'SVR': SVR(), }

# Create Parameter Dictionary for Linear Regression
fit_intercept = [True, False]
normalize = [True, False]
params_LR = dict( fit_intercept = fit_intercept, normalize = normalize)

# Create Parameter Dictionary for SVR
kernel = ['linear', 'rbf', 'poly']
C_list = [10, 100]
ep_list = [0.1, 1, 5]
gamma = [0.01, 0.1]
degree = [2, 3]
params_SVR = dict( kernel = kernel, C = C_list, epsilon = ep_list, gamma = gamma, degree = degree )

# Cross Validation Model
# set k-fold crossvalidation with shuffle
num_fold = 3
seed = 500
kfold = KFold(n_splits=num_fold, shuffle = True, random_state=seed)
import warnings
warnings.filterwarnings('ignore')

In [101]:
# GridSearchCV() -> (a) 
for EST in regression:
    model = regression[EST]
    if (EST == 'LR'):
        params = params_LR
    else:
        params = params_SVR
    grid = GridSearchCV(estimator=model, 
                        n_jobs = 1, 
                        verbose = 10,
                        cv = kfold,
                        scoring = 'neg_mean_squared_error',
                        param_grid = params )
    grid_result = grid.fit(X_train, Y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV 1/3; 1/4] START fit_intercept=True, normalize=True..........................
[CV 1/3; 1/4] END fit_intercept=True, normalize=True;, score=-302.990 total time=   0.0s
[CV 2/3; 1/4] START fit_intercept=True, normalize=True..........................
[CV 2/3; 1/4] END fit_intercept=True, normalize=True;, score=-258.983 total time=   0.0s
[CV 3/3; 1/4] START fit_intercept=True, normalize=True..........................
[CV 3/3; 1/4] END fit_intercept=True, normalize=True;, score=-260.005 total time=   0.0s
[CV 1/3; 2/4] START fit_intercept=True, normalize=False.........................
[CV 1/3; 2/4] END fit_intercept=True, normalize=False;, score=-302.990 total time=   0.0s
[CV 2/3; 2/4] START fit_intercept=True, normalize=False.........................
[CV 2/3; 2/4] END fit_intercept=True, normalize=False;, score=-258.983 total time=   0.0s
[CV 3/3; 2/4] START fit_intercept=True, normalize=False.........................
[CV 3/3

In [ ]:
# Show Best Parameters for both models
print('Best params: ',grid_result.best_params_)
print('Best score: ', grid_result.best_score_)

In [ ]:
# Show Score for each parameter combination for both model
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
 # Display Mean, std, params
grid_result.Bar()

RandomizedSearchCV()

In [ ]:
# Create Model List
regression = { 'LR': LinearRegression(), 'SVR': SVR(), }

In [ ]:
# Create Parameter Dictionary for Linear Regression
fit_intercept = [True, False]
normalize = [True, False]
params_LR = dict( fit_intercept = fit_intercept, normalize = normalize)

In [ ]:
# Create Parameter Dictionary for SVR
kernel = ['linear', 'rbf', 'poly']
C_list = list(np.linspace(0.1, 150, 5, dtype = float))
ep_list = list(np.linspace(0.1, 1, 5, dtype = float))
gamma = list(np.linspace(0.01, 0.1, 5, dtype = float))
degree = [2, 3]
params_SVR = dict( kernel = kernel, C = C_list, epsilon = ep_list, gamma = gamma, degree = degree )

# Cross Validation Model
# set k-fold crossvalidation with shuffle
num_fold = 3
seed = 500
kfold = KFold(n_splits=num_fold, shuffle = True, random_state=seed)
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Show scatter plot compare y_test vs each model prediction
plt.scatter()

In [ ]:
# GridSearchCV() -> (a) 
for EST in regression:
    model = regression[EST]
    if (EST == 'LR'):
        params = params_LR
    else:
        params = params_SVR
    grid = GridSearchCV(estimator=model, 
                        n_jobs = 1, 
                        verbose = 10,
                        cv = kfold,
                        scoring = 'neg_mean_squared_error',
                        param_grid = params )
    grid_result = grid.fit(X_train, Y_train)

In [ ]:
# Show Best Parameters for both models
print('Best params: ',grid_result.best_params_)
print('Best score: ', grid_result.best_score_)

In [ ]:
# Show Score for each parameter combination for both model
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
# Display Mean, std, params
grid_result.Bar()

In [ ]:
# Show scatter plot compare y_test vs each model prediction
plt.scatter()